### Выполним постановку задачи:
$x_{ij}$ - переменная, равная количеству постоянных работников, которые в $i$-ую(утреннюю/ночную) смену в $j$-ый день недели начали свою рабочую неделю, т.е вышли после выходных;<br>
$x_{ijk}$ - переменная, равная количеству временных работников, которые в $i$-ую(утреннюю/ночную) смену в $j$-ый день недели в $k$-ую(первую/вторую) половину смены были привлечены к работе;<br>
$c_{r}$ - недельная плата постоянных работников;<br>
$c_{t}$ - недельная плата временных работников;<br>
Множество $M = \{1, 4, 5, 6, 7\}$ при $j = 1$; $\{1, 2, 5, 6, 7\}$ при $j = 2$; $\{1, 2, 3, 6, 7\}$ при $j = 3$; $\{1, 2, 3, 4, 7\}$ при $j = 4$; $\{1, 2, 3, 4, 5\}$ при $j = 5$; $\{2, 3, 4, 5, 6\}$ при $j = 6$; $\{3, 4, 5, 6, 7\}$ при $j = 7$;

Целевая функция:
$\displaystyle\sum_{i=1}^2\displaystyle\sum_{j=1}^7{c_{r}*x_{ij}} + \displaystyle\sum_{i=1}^2\displaystyle\sum_{j=1}^7\displaystyle\sum_{k=1}^2 {c_{t}*x_{ijk}} \to min$<br>
Ограничения:<br>
$x_{ij} \geq 0$ для всех $i$, для всех $j$<br>

$x_{ijk} \geq 0$ для всех $i$, для всех $j$, для всех $k$<br>

$(\displaystyle\sum_{m \in M}x_{im}) + {x_{ijk}}$ для всех $i$, для всех $j$(помним, что каждому $j$ соответствует своё множество $M$), и для всех $k$<br>

In [580]:
import math
import numpy as np
from cvxopt import matrix, glpk
from prettytable import PrettyTable

import sys

np.set_printoptions(threshold=sys.maxsize)

In [581]:
need = [
    {"morning": [8, 8], "evening": [7, 6]},  # monday
    {"morning": [6, 5], "evening": [4, 5]},  # tuesday
    {"morning": [6, 5], "evening": [4, 4]},  # wednesday
    {"morning": [5, 6], "evening": [6, 7]},  # thursday
    {"morning": [7, 8], "evening": [8, 9]},  # friday
    {"morning": [9, 8], "evening": [7, 6]},  # saturday
    {"morning": [6, 5], "evening": [4, 4]},  # sunday
]
daysToWork = 5
regularMonthPayment = 51200
temporaryHourPayment = 380
weekDays = [
    "Понедельник",
    "Вторник",
    "Среда",
    "Четверг",
    "Пятница",
    "Суббота",
    "Воскресенье",
]

In [582]:
regularPeopleByDay = [
    [1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    #
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
]


In [583]:
temporaryPeopleByDay = np.array([])
amount = len(regularPeopleByDay) * 2
for i in range(amount):
    row = np.concatenate(
        (
            np.zeros(i),
            np.array([1]),
            np.zeros(amount - i - 1),
        )
    )
    temporaryPeopleByDay = (
        np.copy(row) if i == 0 else np.vstack((temporaryPeopleByDay, row))
    )

In [584]:
regularAndTemporaryCombined = []
for i in range(len(regularPeopleByDay)):
    row1 = np.concatenate(
        (np.array(regularPeopleByDay[i]), temporaryPeopleByDay[i * 2])
    )
    row2 = np.concatenate(
        (np.array(regularPeopleByDay[i]), temporaryPeopleByDay[i * 2 + 1])
    )
    shift = np.vstack((row1, row2))
    regularAndTemporaryCombined = (
        np.copy(shift) if i == 0 else np.vstack((regularAndTemporaryCombined, shift))
    )

In [585]:
peopleNeed = []
for i in range(len(need)):
    peopleNeed.extend([need[i]["morning"][0], need[i]["morning"][1]])
for i in range(len(need)):
    peopleNeed.extend([need[i]["evening"][0], need[i]["evening"][1]])


In [586]:
notNegativeAmount = np.eye(len(regularPeopleByDay) + len(temporaryPeopleByDay))


In [587]:
regularWeekPayment = regularMonthPayment / 4
temporaryWeekPayment = temporaryHourPayment * 4

cost = np.concatenate(
    (
        np.full(len(regularPeopleByDay), regularWeekPayment),
        np.full(len(temporaryPeopleByDay), temporaryWeekPayment),
    )
)

In [588]:
targetFunction = matrix(cost, tc="d").T
inequalityLeft = matrix(
    [
        [matrix(-1 * regularAndTemporaryCombined).T],
        [matrix(-1 * notNegativeAmount).T],
    ],
    tc="d",
).T
inequalityRight = matrix(
    [
        [matrix(-1 * np.array(peopleNeed)).T],
        [matrix(np.zeros(len(notNegativeAmount))).T],
    ],
    tc="d",
).T


In [589]:
(status, x) = glpk.ilp(
    c=targetFunction,
    G=inequalityLeft,
    h=inequalityRight,
    I=set(range(len(targetFunction))),
)
result = np.array([*x.T]).astype(int)

In [590]:
def printResult(resultStatus, resultArray):
    print(f"Статус: {resultStatus}")
    if resultStatus == "optimal":
        regularsTable = PrettyTable(["День недели", "7:00 - 15:00", "15:00 - 23:00"])
        for i in range(len(weekDays)):
            regularsTable.add_row(
                [weekDays[i], resultArray[i], resultArray[i + len(weekDays)]]
            )
        print("График выхода постоянных работников:")
        print(regularsTable, end="\n\n")

        temporariesTable = PrettyTable(
            [
                "День недели",
                "7:00 - 11:00",
                "11:00 - 15:00",
                "15:00 - 19:00",
                "19:00 - 23:00",
            ]
        )
        for i in range(len(weekDays)):
            index_morning = len(regularPeopleByDay) + i * 2
            index_evening = len(regularPeopleByDay) + len(weekDays) * 2 + i * 2
            temporariesTable.add_row(
                [
                    weekDays[i],
                    resultArray[index_morning],
                    resultArray[index_morning + 1],
                    resultArray[index_evening],
                    resultArray[index_evening + 1],
                ]
            )
        print("График выхода временных работников:")
        print(temporariesTable, end="\n\n")
    else:
        print("Оптимальное решение не было найдено.")

In [591]:
def amountOfWorkersPresent(rows, col):
    workersByHours = np.matmul(rows, col).astype(int)
    workersTable = PrettyTable(
            [
                "День недели",
                "7:00 - 11:00",
                "11:00 - 15:00",
                "15:00 - 19:00",
                "19:00 - 23:00",
            ]
        )
    for i in range(len(weekDays)):
        index_morning = i * 2
        index_evening = len(weekDays) * 2 + i * 2
        workersTable.add_row(
            [
                weekDays[i],
                workersByHours[index_morning],
                workersByHours[index_morning + 1],
                workersByHours[index_evening],
                workersByHours[index_evening + 1],
            ]
        )
    print("Количество работников на каждые 4 часа:")
    print(workersTable)

In [592]:
printResult(status, result)
amountOfWorkersPresent(regularAndTemporaryCombined, result)

Статус: optimal
График выхода постоянных работников:
+-------------+--------------+---------------+
| День недели | 7:00 - 15:00 | 15:00 - 23:00 |
+-------------+--------------+---------------+
| Понедельник |      0       |       3       |
|   Вторник   |      3       |       0       |
|    Среда    |      1       |       1       |
|   Четверг   |      1       |       2       |
|   Пятница   |      2       |       1       |
|   Суббота   |      1       |       0       |
| Воскресенье |      0       |       0       |
+-------------+--------------+---------------+

График выхода временных работников:
+-------------+--------------+---------------+---------------+---------------+
| День недели | 7:00 - 11:00 | 11:00 - 15:00 | 15:00 - 19:00 | 19:00 - 23:00 |
+-------------+--------------+---------------+---------------+---------------+
| Понедельник |      4       |       4       |       1       |       0       |
|   Вторник   |      0       |       0       |       0       |       1       